# Ejemplo sencillo con Spark SQL


Este notebook muestra un ejemplo básico de uso de **Spark SQL** con un archivo CSV de reseñas.
Se carga el archivo `ejercicio_reviews_simple.csv`, se crea una vista temporal y se ejecutan consultas SQL.


## 1) Crear SparkSession

In [1]:

import os, sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import size, split

# Configuración para asegurar mismo Python en driver y workers (Windows/Colab)
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

spark = (SparkSession.builder
         .appName("Ejemplo_SparkSQL_Reviews")
         .config("spark.python.use.daemon","false")
         .config("spark.python.worker.reuse","false")
         .getOrCreate())

spark


## 2) Cargar archivo CSV

In [2]:

csv_path = "ejercicio_reviews_simple.csv"

df = (spark.read
      .option("header", True)
      .option("inferSchema", True)
      .csv(csv_path))

df.show(truncate=False)
df.printSchema()


+---+-----------------------------------------+
|id |texto                                    |
+---+-----------------------------------------+
|1  |Muy buen servicio y atención rápida.     |
|2  |Producto defectuoso, no funcionó.        |
|3  |Excelente calidad, volvería a comprar.   |
|4  |Entrega lenta, pero producto aceptable.  |
|5  |El empaque llegó dañado.                 |
|6  |Me encantó, lo recomiendo totalmente.    |
|7  |No cumplió con mis expectativas.         |
|8  |Buen diseño y fácil de usar.             |
|9  |Precio demasiado alto para la calidad.   |
|10 |Superó mis expectativas, cinco estrellas.|
+---+-----------------------------------------+

root
 |-- id: integer (nullable = true)
 |-- texto: string (nullable = true)



## 3) Crear vista temporal con número de palabras

In [3]:

# Agregar columna num_palabras usando funciones nativas de Spark
df = df.withColumn("num_palabras", size(split(df["texto"], " ")))

# Registrar vista temporal
df.createOrReplaceTempView("reviews")


## 4) Consultas con Spark SQL

In [4]:

# Todas las reseñas
spark.sql("SELECT * FROM reviews").show(truncate=False)

# Total de reseñas
spark.sql("SELECT COUNT(*) as total_resenas FROM reviews").show()

# Promedio de palabras por reseña
spark.sql("SELECT AVG(num_palabras) as promedio_palabras FROM reviews").show()


+---+-----------------------------------------+------------+
|id |texto                                    |num_palabras|
+---+-----------------------------------------+------------+
|1  |Muy buen servicio y atención rápida.     |6           |
|2  |Producto defectuoso, no funcionó.        |4           |
|3  |Excelente calidad, volvería a comprar.   |5           |
|4  |Entrega lenta, pero producto aceptable.  |5           |
|5  |El empaque llegó dañado.                 |4           |
|6  |Me encantó, lo recomiendo totalmente.    |5           |
|7  |No cumplió con mis expectativas.         |5           |
|8  |Buen diseño y fácil de usar.             |6           |
|9  |Precio demasiado alto para la calidad.   |6           |
|10 |Superó mis expectativas, cinco estrellas.|5           |
+---+-----------------------------------------+------------+

+-------------+
|total_resenas|
+-------------+
|           10|
+-------------+

+-----------------+
|promedio_palabras|
+-----------------+
|   